In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from regime_ml.utils import load_configs
from regime_ml.data import select_data
from regime_ml.data import load_dataframe
from regime_ml.data import load_raw_data
import pandas as pd

In [19]:
macro_cfg = load_configs()["macro_data"]["regime_universe"]

In [20]:
df_macro_raw = load_raw_data(macro_cfg["parquet_path"], macro_cfg["output_path"])

Found 5 parquet files in /Users/alecmitchell-thomson/Desktop/Coding/quant-data/macro
Files to combine:
  - employment/employment.parquet
  - growth/growth.parquet
  - inflation/inflation.parquet
  - liquidity/liquidity.parquet
  - rates/rates.parquet


Loading parquet files: 100%|██████████| 5/5 [00:00<00:00, 278.25it/s]


Combining dataframes...
Combined dataframe shape: (100466, 5)
Columns: ['series_code', 'date', 'value', 'series_name', 'category']
Categories: ['rates', 'employment', 'growth', 'inflation', 'liquidity']
Unique series: 39

Saving to: /Users/alecmitchell-thomson/Desktop/Coding/regime-conditioned-equity-ml/data/raw/macro_raw.parquet
✓ Successfully saved 100,466 rows to /Users/alecmitchell-thomson/Desktop/Coding/regime-conditioned-equity-ml/data/raw/macro_raw.parquet


In [21]:
df_macro_raw = load_dataframe(macro_cfg["output_path"])
df_selected = select_data(df_macro_raw, macro_cfg)
df_selected[df_selected["series_code"] == "INDPRO"]

,series_code,date,value,series_name,category
14692,INDPRO,2000-01-01,91.5380,industrial_production,growth
14693,INDPRO,2000-02-01,91.8239,industrial_production,growth
14694,INDPRO,2000-03-01,92.1504,industrial_production,growth
14695,INDPRO,2000-04-01,92.6989,industrial_production,growth
14696,INDPRO,2000-05-01,92.9499,industrial_production,growth
...,...,...,...,...,...
14999,INDPRO,2025-08-01,101.5867,industrial_production,growth
15000,INDPRO,2025-09-01,101.7779,industrial_production,growth
15001,INDPRO,2025-10-01,101.5163,industrial_production,growth
15002,INDPRO,2025-11-01,101.9528,industrial_production,growth


In [22]:
from regime_ml.data import roll_weekend_releases
df_selected = roll_weekend_releases(df_selected)
df_selected

,series_code,date,value,series_name,category
0,CFNAI,2000-01-03,0.15,chicago_fed_national_activity_index,growth
1,CFNAI,2000-02-01,-0.23,chicago_fed_national_activity_index,growth
2,CFNAI,2000-03-01,0.43,chicago_fed_national_activity_index,growth
3,CFNAI,2000-04-03,0.42,chicago_fed_national_activity_index,growth
4,CFNAI,2000-05-01,-0.35,chicago_fed_national_activity_index,growth
...,...,...,...,...,...
29756,VIXCLS,2026-01-09,14.49,vix_volatility_index,growth
29757,VIXCLS,2026-01-12,15.12,vix_volatility_index,growth
29758,VIXCLS,2026-01-13,15.98,vix_volatility_index,growth
29759,VIXCLS,2026-01-14,16.75,vix_volatility_index,growth


In [23]:
from regime_ml.data import create_master_calendar
master_calendar = create_master_calendar(
    start_date=macro_cfg["lookback"]["start"],
    end_date=macro_cfg["lookback"]["end"]
    )
master_calendar

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05', '2000-01-06',
               '2000-01-07', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14',
               ...
               '2026-01-02', '2026-01-05', '2026-01-06', '2026-01-07',
               '2026-01-08', '2026-01-09', '2026-01-12', '2026-01-13',
               '2026-01-14', '2026-01-15'],
              dtype='datetime64[ns]', length=6529, freq='C')

In [24]:
df_selected["date"] = pd.to_datetime(df_selected["date"])

In [25]:
aligned_series = []
for series_code in df_selected["series_code"].unique():
    series_df = df_selected[df_selected["series_code"] == series_code]

    aligned = series_df.set_index("date").reindex(master_calendar, method="ffill").reset_index()
    aligned["series_code"] = series_code
    aligned_series.append(aligned)

df_aligned = pd.concat(aligned_series).reset_index(drop=True).rename(columns={"index": "date"})
df_aligned

,date,series_code,value,series_name,category
0,2000-01-03,CFNAI,0.15,chicago_fed_national_activity_index,growth
1,2000-01-04,CFNAI,0.15,chicago_fed_national_activity_index,growth
2,2000-01-05,CFNAI,0.15,chicago_fed_national_activity_index,growth
3,2000-01-06,CFNAI,0.15,chicago_fed_national_activity_index,growth
4,2000-01-07,CFNAI,0.15,chicago_fed_national_activity_index,growth
...,...,...,...,...,...
58756,2026-01-09,VIXCLS,14.49,vix_volatility_index,growth
58757,2026-01-12,VIXCLS,15.12,vix_volatility_index,growth
58758,2026-01-13,VIXCLS,15.98,vix_volatility_index,growth
58759,2026-01-14,VIXCLS,16.75,vix_volatility_index,growth


In [26]:
df_aligned.groupby("series_code").apply(lambda x: x.isna().sum())

/var/folders/7m/_v42_c3x3pxc0myzbkfdk97h0000gn/T/ipykernel_17216/3721550661.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_aligned.groupby("series_code").apply(lambda x: x.isna().sum())


,date,series_code,value,series_name,category
series_code,,,,,
CFNAI,0,0,0,0,0
DGS10,0,0,0,0,0
DGS2,0,0,0,0,0
ICSA,0,0,0,0,0
INDPRO,0,0,0,0,0
NFCI,0,0,4,4,4
PCEPILFE,0,0,0,0,0
T10Y3M,0,0,0,0,0
VIXCLS,0,0,0,0,0


In [27]:
df_aligned[df_aligned["series_code"] == "NFCI"]

,date,series_code,value,series_name,category
32645,2000-01-03,NFCI,NaN,NaN,NaN
32646,2000-01-04,NFCI,NaN,NaN,NaN
32647,2000-01-05,NFCI,NaN,NaN,NaN
32648,2000-01-06,NFCI,NaN,NaN,NaN
32649,2000-01-07,NFCI,-0.18231,chicago_fed_financial_conditions,liquidity
...,...,...,...,...,...
39169,2026-01-09,NFCI,-0.57258,chicago_fed_financial_conditions,liquidity
39170,2026-01-12,NFCI,-0.57258,chicago_fed_financial_conditions,liquidity
39171,2026-01-13,NFCI,-0.57258,chicago_fed_financial_conditions,liquidity
39172,2026-01-14,NFCI,-0.57258,chicago_fed_financial_conditions,liquidity
